In [ ]:
!pip install openai pinecone-client tiktoken

import os
import torch
from openai import OpenAI
from pinecone import Pinecone
import tiktoken
from transformers import AutoTokenizer, AutoModel


In [ ]:
# Initialize the tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained("intfloat/e5-small-v2")
model = AutoModel.from_pretrained("intfloat/e5-small-v2")

# API Keys (replace with your actual keys)
os.environ["OPENAI_API_KEY"] = "sk-proj"
PINECONE_API_KEY = "PINECONE_API_"

# Variables
INDEX_NAME = "ragchatbot"
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"
MAX_TOKENS = 225

In [ ]:
# Initialize OpenAI client
client = OpenAI()

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)


index = pc.Index(INDEX_NAME)

# Initialize tokenizer
tokenizer = tiktoken.encoding_for_model(GPT_MODEL)

In [ ]:
# Cell 3: Store conversation history in an array
conversation = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you! How can I assist you today?",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.",
    "20: User: Thanks for the tips! Any last advice? | Bot: Just enjoy your journey and make the most out of your experiences."
]


In [ ]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Function to count tokens
def count_tokens(text):
    return len(tokenizer.encode(text))

# Function to store chat history in Pinecone in chunks
def store_chat_history(conversation, batch_size=10):
    chunk = []
    for entry in conversation:
        parts = entry.split(": ", 2)
        if len(parts) == 3:
            id, user_message, bot_message = parts
            full_message = f"User: {user_message} Bot: {bot_message}"
            embedding = get_embedding(full_message).tolist()
            chunk.append((id, embedding, {"text": full_message}))

            # When chunk reaches batch size, upsert into Pinecone
            if len(chunk) == batch_size:
                index.upsert(vectors=chunk)
                chunk = []

    # Upsert any remaining vectors in the final chunk
    if chunk:
        index.upsert(vectors=chunk)

# Store the conversation history
store_chat_history(conversation)

# Cell 5: Define the RAG mechanism
def fetch_relevant_messages(query, top_k=3):
    query_embedding = get_embedding(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    print(results)
    return [match['metadata']['text'] for match in results['matches']]




In [ ]:
#Retrive from pinecone
def test_retrieve_and_print_vectors(query):
    relevant_history_with_vectors = fetch_relevant_messages(query)

    print("Relevant History and Vectors:")
    for text in relevant_history_with_vectors:
        print(f"Text: {text}")
        print("------")

# Test the function with a query
test_query = "Do you think it will help me stay fit?"
test_retrieve_and_print_vectors(test_query)



In [ ]:
# Function to generate response using relevant history
def generate_response(query, relevant_history):
    context = "\n".join(relevant_history)
    prompt = f"Context:\n{context}\n\nQuery: {query}\nResponse:"

    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=MAX_TOKENS - count_tokens(prompt)
    )
    return response.choices[0].message.content

# Function to process user input
def process_user_input(user_input):
    relevant_history = fetch_relevant_messages(user_input)
    response = generate_response(user_input, relevant_history)
    return relevant_history, response

# Cell 6: Testing the implementation

def test_final_prompt():
    final_test_prompt = "What was that book recommendation again?"
    relevant_history, response = process_user_input(final_test_prompt)

    print(f"Final Test Prompt: {final_test_prompt}")
    print(f"Context Referred: {relevant_history}")
    print(f"Final Test Prompt Response: {response}")

    total_tokens = count_tokens("\n".join(relevant_history) + response)
    print(f"\nTotal tokens (context + response): {total_tokens}")

# Call the test function to generate the Final Test Prompt Response
test_final_prompt()